In [1]:
import pandas as pd
import pulp

In [2]:
 df = pd.read_csv('Optimization.csv')

In [3]:
df.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Starting SOC,NaN,NaN,NaN,NaN,NaN
2,NaN,100% (10 MWh),NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Date/Time,Hour Ending,WFTANK Real-Time LMP,WFTANK Day-Ahead LMP,RRS (spinning reserve),REGUP
6,NaN,9/10/2023,1,$ 22.58,$ 21.49,$ 3.50,$ 2.28
7,NaN,9/10/2023,2,$ 21.94,$ 19.65,$ 2.54,$ 1.32
8,NaN,9/10/2023,3,$ 24.59,$ 18.71,$ 1.50,$ 1.42
9,NaN,9/10/2023,4,$ 27.43,$ 18.44,$ 1.50,$ 1.46


In [4]:
# Drop the first 4 rows
df = df.drop([0, 1, 2, 3, 4])

# Reset the index
df = df.reset_index(drop=True)

# Set the 5th row as the header
df.columns = df.iloc[0]

# Drop the new header row
df = df.drop(df.index[0])

# Reset the index again to have a clean DataFrame
df = df.reset_index(drop=True)

# Drop columns with NaN labels
df = df.loc[:, ~df.columns.isna()]

# Define the function to remove non-numeric characters except the decimal point and handle negative values in brackets
def clean_numeric_data(column):
    # Replace values in brackets with negative values
    column = column.replace(r'\((.*?)\)', r'-\1', regex=True)
    # Remove all non-numeric characters except the decimal point and minus sign
    column = column.replace('[^\d.-]', '', regex=True)
    return column

# Specify the columns to clean
columns_to_clean = ['WFTANK Real-Time LMP', 'WFTANK Day-Ahead LMP', 'RRS (spinning reserve)', 'REGUP']

# Apply the cleaning function to each specified column
for column in columns_to_clean:
    df[column] = clean_numeric_data(df[column])

# Convert cleaned columns to numeric
for column in columns_to_clean:
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [5]:
df.head()

,Date/Time,Hour Ending,WFTANK Real-Time LMP,WFTANK Day-Ahead LMP,RRS (spinning reserve),REGUP
0,9/10/2023,1,22.58,21.49,3.50,2.28
1,9/10/2023,2,21.94,19.65,2.54,1.32
2,9/10/2023,3,24.59,18.71,1.50,1.42
3,9/10/2023,4,27.43,18.44,1.50,1.46
4,9/10/2023,5,21.78,18.72,1.50,1.50


### Linear Programming for PT1

In [6]:
# Convert price columns to numeric values
df['WFTANK Real-Time LMP'] = pd.to_numeric(df['WFTANK Real-Time LMP'], errors='coerce')
df['WFTANK Day-Ahead LMP'] = pd.to_numeric(df['WFTANK Day-Ahead LMP'], errors='coerce')
df['RRS (spinning reserve)'] = pd.to_numeric(df['RRS (spinning reserve)'], errors='coerce')

# Define the battery parameters
battery_capacity_mwh = 10
battery_initial_soc = 10  # MWh, 100% SOC

# Define the time period
T = len(df)

# Create a linear programming problem
lp_problem = pulp.LpProblem("Battery_Optimization", pulp.LpMaximize)

# Decision variables
charge = pulp.LpVariable.dicts("charge", range(T), lowBound=0, upBound=battery_capacity_mwh)
discharge = pulp.LpVariable.dicts("discharge", range(T), lowBound=0, upBound=battery_capacity_mwh)
reserves = pulp.LpVariable.dicts("reserves", range(T), lowBound=0, upBound=battery_capacity_mwh)
soc = pulp.LpVariable.dicts("soc", range(T+1), lowBound=0, upBound=battery_capacity_mwh)

# Initial SOC
lp_problem += soc[0] == battery_initial_soc

# Objective function
lp_problem += pulp.lpSum(
    -df.loc[t, 'WFTANK Real-Time LMP'] * charge[t] +
    df.loc[t, 'WFTANK Real-Time LMP'] * discharge[t] +
    df.loc[t, 'RRS (spinning reserve)'] * reserves[t]
    for t in range(T)
)

# Constraints
for t in range(T):
    lp_problem += soc[t + 1] == soc[t] + charge[t] - discharge[t] - reserves[t]
    lp_problem += charge[t] + discharge[t] + reserves[t] <= battery_capacity_mwh

# Solve the problem
lp_problem.solve()

# Retrieve the results
df['Charge (MWh)'] = [charge[t].varValue for t in range(T)]
df['Discharge (MWh)'] = [discharge[t].varValue for t in range(T)]
df['Spinning Reserves (MW)'] = [reserves[t].varValue for t in range(T)]
df['SOC (MWh)'] = [soc[t+1].varValue for t in range(T)]

# Calculate profit
df['Profit ($)'] = (
    -df['WFTANK Real-Time LMP'] * df['Charge (MWh)'] +
    df['WFTANK Real-Time LMP'] * df['Discharge (MWh)'] +
    df['RRS (spinning reserve)'] * df['Spinning Reserves (MW)']
)

total_profit = df['Profit ($)'].sum()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/karthiksvasisht/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0d/r6ls8rfx1sjb9p8mlgmzld6w0000gn/T/5a63847b263d42638e7fc20a50d01965-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/0d/r6ls8rfx1sjb9p8mlgmzld6w0000gn/T/5a63847b263d42638e7fc20a50d01965-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 438 COLUMNS
At line 2816 RHS
At line 3250 BOUNDS
At line 4116 ENDATA
Problem MODEL has 433 rows, 865 columns and 1729 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 430 (-3) rows, 645 (-220) columns and 1289 (-440) elements
0  Obj 225.8 Dual inf 6377.0398 (221)
83  Obj 63996.2 Primal inf 34240 (218)
166  Obj 31867.9 Primal inf 2279.9998 (166)
249  Obj 20159.5 Primal inf 1489.9999 (122)
332  Obj 13862 Primal inf 869.99992 (85)
415  Obj 12604.6

In [7]:
# Display the total profit
print(f"Total Profit: ${total_profit:.2f}")

df.head(50)

Total Profit: $12196.55


,Date/Time,Hour Ending,WFTANK Real-Time LMP,WFTANK Day-Ahead LMP,RRS (spinning reserve),REGUP,Charge (MWh),Discharge (MWh),Spinning Reserves (MW),SOC (MWh),Profit ($)
0,9/10/2023,1,22.58,21.49,3.50,2.28,0.0,10.0,0.0,0.0,225.80
1,9/10/2023,2,21.94,19.65,2.54,1.32,10.0,0.0,0.0,10.0,-219.40
2,9/10/2023,3,24.59,18.71,1.50,1.42,5.0,5.0,0.0,10.0,0.00
3,9/10/2023,4,27.43,18.44,1.50,1.46,0.0,10.0,0.0,0.0,274.30
4,9/10/2023,5,21.78,18.72,1.50,1.50,0.0,0.0,0.0,0.0,0.00
5,9/10/2023,6,18.73,19.99,2.40,2.40,5.0,5.0,0.0,0.0,0.00
6,9/10/2023,7,18.28,18.88,1.69,1.69,5.0,5.0,0.0,0.0,0.00
7,9/10/2023,8,16.98,18.45,1.50,1.50,5.0,5.0,0.0,0.0,0.00
8,9/10/2023,9,15.45,17.41,1.50,1.50,0.0,0.0,0.0,0.0,0.00
9,9/10/2023,10,14.45,16.66,3.40,3.40,10.0,0.0,0.0,10.0,-144.50


In [8]:
# Convert price columns to numeric values
df['WFTANK Real-Time LMP'] = pd.to_numeric(df['WFTANK Real-Time LMP'], errors='coerce')
df['WFTANK Day-Ahead LMP'] = pd.to_numeric(df['WFTANK Day-Ahead LMP'], errors='coerce')
df['RRS (spinning reserve)'] = pd.to_numeric(df['RRS (spinning reserve)'], errors='coerce')

# Define the battery parameters
battery_capacity_mwh = 10
battery_initial_soc = 10  # MWh, 100% SOC

# Define the time period
T = len(df)

# Create a linear programming problem
lp_problem = pulp.LpProblem("Battery_Optimization", pulp.LpMaximize)

# Decision variables
charge = pulp.LpVariable.dicts("charge", range(T), lowBound=0, upBound=battery_capacity_mwh)
discharge = pulp.LpVariable.dicts("discharge", range(T), lowBound=0, upBound=battery_capacity_mwh)
reserves = pulp.LpVariable.dicts("reserves", range(T), lowBound=0, upBound=battery_capacity_mwh)
soc = pulp.LpVariable.dicts("soc", range(T+1), lowBound=0, upBound=battery_capacity_mwh)

# Initial SOC constraint
lp_problem += soc[0] == battery_initial_soc, "Initial_SOC"

# Objective function
lp_problem += pulp.lpSum(
    -df.loc[t, 'WFTANK Real-Time LMP'] * charge[t] +
    df.loc[t, 'WFTANK Real-Time LMP'] * discharge[t] +
    df.loc[t, 'RRS (spinning reserve)'] * reserves[t]
    for t in range(T)
), "Total_Profit"

# Constraints
for t in range(T):
    lp_problem += soc[t + 1] == soc[t] + charge[t] - discharge[t] - reserves[t], f"SOC_Update_{t}"
    lp_problem += charge[t] + discharge[t] + reserves[t] <= battery_capacity_mwh, f"Capacity_Limit_{t}"

# Solve the problem
lp_problem.solve()

# Check the status of the solution
print(f"Solver Status: {pulp.LpStatus[lp_problem.status]}")

# Retrieve the results
df['Charge (MWh)'] = [charge[t].varValue for t in range(T)]
df['Discharge (MWh)'] = [discharge[t].varValue for t in range(T)]
df['Spinning Reserves (MW)'] = [reserves[t].varValue for t in range(T)]
df['SOC (MWh)'] = [soc[t+1].varValue for t in range(T)]

# Calculate profit
df['Profit ($)'] = (
    -df['WFTANK Real-Time LMP'] * df['Charge (MWh)'] +
    df['WFTANK Real-Time LMP'] * df['Discharge (MWh)'] +
    df['RRS (spinning reserve)'] * df['Spinning Reserves (MW)']
)

total_profit = df['Profit ($)'].sum()

# Display the total profit
print(f"Total Profit: ${total_profit:.2f}")

# Group by date and calculate the profit for each date
df['Date'] = pd.to_datetime(df['Date/Time']).dt.date
daily_profit = df.groupby('Date')['Profit ($)'].sum().reset_index()

print(daily_profit)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/karthiksvasisht/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0d/r6ls8rfx1sjb9p8mlgmzld6w0000gn/T/0acff836a8064e078054a3230bd7089c-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/0d/r6ls8rfx1sjb9p8mlgmzld6w0000gn/T/0acff836a8064e078054a3230bd7089c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 438 COLUMNS
At line 2816 RHS
At line 3250 BOUNDS
At line 4116 ENDATA
Problem MODEL has 433 rows, 865 columns and 1729 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 430 (-3) rows, 645 (-220) columns and 1289 (-440) elements
0  Obj 225.8 Dual inf 6377.0398 (221)
83  Obj 63996.2 Primal inf 34240 (218)
166  Obj 31867.9 Primal inf 2279.9998 (166)
249  Obj 20159.5 Primal inf 1489.9999 (122)
332  Obj 13862 Primal inf 869.99992 (85)
415  Obj 12604.6

In [9]:
df.head(50)

,Date/Time,Hour Ending,WFTANK Real-Time LMP,WFTANK Day-Ahead LMP,RRS (spinning reserve),REGUP,Charge (MWh),Discharge (MWh),Spinning Reserves (MW),SOC (MWh),Profit ($),Date
0,9/10/2023,1,22.58,21.49,3.50,2.28,0.0,10.0,0.0,0.0,225.80,2023-09-10
1,9/10/2023,2,21.94,19.65,2.54,1.32,10.0,0.0,0.0,10.0,-219.40,2023-09-10
2,9/10/2023,3,24.59,18.71,1.50,1.42,5.0,5.0,0.0,10.0,0.00,2023-09-10
3,9/10/2023,4,27.43,18.44,1.50,1.46,0.0,10.0,0.0,0.0,274.30,2023-09-10
4,9/10/2023,5,21.78,18.72,1.50,1.50,0.0,0.0,0.0,0.0,0.00,2023-09-10
5,9/10/2023,6,18.73,19.99,2.40,2.40,5.0,5.0,0.0,0.0,0.00,2023-09-10
6,9/10/2023,7,18.28,18.88,1.69,1.69,5.0,5.0,0.0,0.0,0.00,2023-09-10
7,9/10/2023,8,16.98,18.45,1.50,1.50,5.0,5.0,0.0,0.0,0.00,2023-09-10
8,9/10/2023,9,15.45,17.41,1.50,1.50,0.0,0.0,0.0,0.0,0.00,2023-09-10
9,9/10/2023,10,14.45,16.66,3.40,3.40,10.0,0.0,0.0,10.0,-144.50,2023-09-10


### Linear Programming for PT2

In [11]:
# Define the battery parameters
battery_size_mw = 10
battery_capacity_mwh = 10
initial_soc = 10  # MWh, 100% SOC

# Add new columns for decision making
df['Charge (MWh)'] = 0
df['Discharge (MWh)'] = 0
df['Spinning Reserves (MW)'] = 0
df['Up Regulation (MW)'] = 0
df['SOC (MWh)'] = initial_soc
df['Profit ($)'] = 0

# Function to optimize the battery usage for each hour
def optimize_battery(df):
    soc = initial_soc
    total_profit = 0
    for i in range(len(df)):
        # Get prices for the current hour
        real_time_price = float(df.loc[i, 'WFTANK Real-Time LMP'])
        day_ahead_price = float(df.loc[i, 'WFTANK Day-Ahead LMP'])
        spinning_reserve_price = float(df.loc[i, 'RRS (spinning reserve)'])
        up_regulation_price = float(df.loc[i, 'REGUP'])
        
        # Initialize decisions for the hour
        charge_amount = 0
        discharge_amount = 0
        spinning_reserves = 0
        up_regulation = 0
        profit = 0
        
        # Decide to sell spinning reserves if possible
        if spinning_reserve_price > 0 and soc >= 1:
            spinning_reserves = min(battery_size_mw, soc)
            df.at[i, 'Spinning Reserves (MW)'] = spinning_reserves
            profit += spinning_reserve_price * spinning_reserves
        
        # Decide to sell up regulation if possible
        if up_regulation_price > 0 and soc >= 0.2 * battery_size_mw:
            up_regulation = min(battery_size_mw, soc / 0.2)  # Ensure enough SOC for 0.2 MWh per 1 MW
            df.at[i, 'Up Regulation (MW)'] = up_regulation
            soc -= 0.2 * up_regulation  # Update SOC for energy discharged for Up Regulation
            profit += up_regulation_price * up_regulation
            profit += real_time_price * 0.2 * up_regulation  # Payment at real-time price for energy discharged
        
        # Charge when real-time price is low
        if real_time_price < day_ahead_price and soc < battery_capacity_mwh:
            charge_amount = min(battery_size_mw, battery_capacity_mwh - soc)
            df.at[i, 'Charge (MWh)'] = charge_amount
            soc += charge_amount
            profit -= real_time_price * charge_amount
        
        # Discharge when real-time price is high
        elif real_time_price > day_ahead_price and soc > 0:
            discharge_amount = min(battery_size_mw, soc)
            df.at[i, 'Discharge (MWh)'] = discharge_amount
            soc -= discharge_amount
            profit += real_time_price * discharge_amount
        
        # Ensure SOC is within valid range
        soc = max(0, min(soc, battery_capacity_mwh))
        
        # Update SOC and profit
        df.at[i, 'SOC (MWh)'] = soc
        df.at[i, 'Profit ($)'] = profit
        total_profit += profit
        
        # Print the detailed calculations and decisions for the current row
        print(f"Hour: {df.loc[i, 'Hour Ending']}, Real-Time Price: {real_time_price}, Day-Ahead Price: {day_ahead_price}, Spinning Reserve Price: {spinning_reserve_price}, Up Regulation Price: {up_regulation_price}")
        print(f"Charge Amount: {charge_amount}, Discharge Amount: {discharge_amount}, Spinning Reserves: {spinning_reserves}, Up Regulation: {up_regulation}")
        print(f"State of Charge (SOC): {soc}, Profit: {profit}, Total Profit: {total_profit}\n")
    
    return df, total_profit

# Apply the optimization function to the DataFrame
df, total_profit = optimize_battery(df)

# Display the total profit
print(f"Total Profit: ${total_profit:.2f}")

# Calculate and display the profits for each date
df['Date'] = pd.to_datetime(df['Date/Time']).dt.date
daily_profit = df.groupby('Date')['Profit ($)'].sum().reset_index()

print(daily_profit)

Hour: 1, Real-Time Price: 22.58, Day-Ahead Price: 21.49, Spinning Reserve Price: 3.5, Up Regulation Price: 2.28
Charge Amount: 0, Discharge Amount: 8.0, Spinning Reserves: 10, Up Regulation: 10
State of Charge (SOC): 0, Profit: 283.59999999999997, Total Profit: 283.59999999999997

Hour: 2, Real-Time Price: 21.94, Day-Ahead Price: 19.65, Spinning Reserve Price: 2.54, Up Regulation Price: 1.32
Charge Amount: 0, Discharge Amount: 0, Spinning Reserves: 0, Up Regulation: 0
State of Charge (SOC): 0, Profit: 0, Total Profit: 283.59999999999997

Hour: 3, Real-Time Price: 24.59, Day-Ahead Price: 18.71, Spinning Reserve Price: 1.5, Up Regulation Price: 1.42
Charge Amount: 0, Discharge Amount: 0, Spinning Reserves: 0, Up Regulation: 0
State of Charge (SOC): 0, Profit: 0, Total Profit: 283.59999999999997

Hour: 4, Real-Time Price: 27.43, Day-Ahead Price: 18.44, Spinning Reserve Price: 1.5, Up Regulation Price: 1.46
Charge Amount: 0, Discharge Amount: 0, Spinning Reserves: 0, Up Regulation: 0
State

In [16]:
df.head(20)

,Date/Time,Hour Ending,WFTANK Real-Time LMP,WFTANK Day-Ahead LMP,RRS (spinning reserve),REGUP,Charge (MWh),Discharge (MWh),Spinning Reserves (MW),SOC (MWh),Profit ($),Date,Up Regulation (MW)
0,9/10/2023,1,22.58,21.49,3.50,2.28,0,8,10,0,283.6,2023-09-10,10
1,9/10/2023,2,21.94,19.65,2.54,1.32,0,0,0,0,0.0,2023-09-10,0
2,9/10/2023,3,24.59,18.71,1.50,1.42,0,0,0,0,0.0,2023-09-10,0
3,9/10/2023,4,27.43,18.44,1.50,1.46,0,0,0,0,0.0,2023-09-10,0
4,9/10/2023,5,21.78,18.72,1.50,1.50,0,0,0,0,0.0,2023-09-10,0
5,9/10/2023,6,18.73,19.99,2.40,2.40,10,0,0,10,-187.3,2023-09-10,0
6,9/10/2023,7,18.28,18.88,1.69,1.69,2,0,10,10,33.8,2023-09-10,10
7,9/10/2023,8,16.98,18.45,1.50,1.50,2,0,10,10,30.0,2023-09-10,10
8,9/10/2023,9,15.45,17.41,1.50,1.50,2,0,10,10,30.0,2023-09-10,10
9,9/10/2023,10,14.45,16.66,3.40,3.40,2,0,10,10,68.0,2023-09-10,10


In [17]:
import pandas as pd

# Define the battery parameters
battery_size_mw = 10
battery_capacity_mwh = 10
initial_soc = 10  # MWh, 100% SOC

# Add new columns for decision making
df['Charge (MWh)'] = 0
df['Discharge (MWh)'] = 0
df['Spinning Reserves (MW)'] = 0
df['Up Regulation (MW)'] = 0
df['SOC (MWh)'] = initial_soc
df['Profit ($)'] = 0

# Function to optimize the battery usage for each hour
def optimize_battery(df):
    soc = initial_soc
    total_profit = 0
    for i in range(len(df)):
        # Get prices for the current hour
        real_time_price = float(df.loc[i, 'WFTANK Real-Time LMP'])
        day_ahead_price = float(df.loc[i, 'WFTANK Day-Ahead LMP'])
        spinning_reserve_price = float(df.loc[i, 'RRS (spinning reserve)'])
        up_regulation_price = float(df.loc[i, 'REGUP'])
        
        # Initialize decisions for the hour
        charge_amount = 0
        discharge_amount = 0
        spinning_reserves = 0
        up_regulation = 0
        profit = 0

        # Decide to sell up regulation if possible
        if up_regulation_price > 0 and soc >= 0.2 * battery_size_mw:
            up_regulation = min(battery_size_mw, soc / 0.2)  # Ensure enough SOC for 0.2 MWh per 1 MW
            df.at[i, 'Up Regulation (MW)'] = up_regulation
            soc -= 0.2 * up_regulation  # Update SOC for energy discharged for Up Regulation
            profit += up_regulation_price * up_regulation
            profit += real_time_price * 0.2 * up_regulation  # Payment at real-time price for energy discharged
        
        # Decide to sell spinning reserves if possible
        if up_regulation == 0 and spinning_reserve_price > 0 and soc >= 1:
            spinning_reserves = min(battery_size_mw, soc)
            df.at[i, 'Spinning Reserves (MW)'] = spinning_reserves
            soc -= spinning_reserves
            profit += spinning_reserve_price * spinning_reserves

        # Charge when real-time price is low
        if up_regulation == 0 and spinning_reserves == 0 and real_time_price < day_ahead_price and soc < battery_capacity_mwh:
            charge_amount = min(battery_size_mw, battery_capacity_mwh - soc)
            df.at[i, 'Charge (MWh)'] = charge_amount
            soc += charge_amount
            profit -= real_time_price * charge_amount
        
        # Discharge when real-time price is high
        elif up_regulation == 0 and spinning_reserves == 0 and real_time_price > day_ahead_price and soc > 0:
            discharge_amount = min(battery_size_mw, soc)
            df.at[i, 'Discharge (MWh)'] = discharge_amount
            soc -= discharge_amount
            profit += real_time_price * discharge_amount
        
        # Ensure SOC is within valid range
        soc = max(0, min(soc, battery_capacity_mwh))
        
        # Update SOC and profit
        df.at[i, 'SOC (MWh)'] = soc
        df.at[i, 'Profit ($)'] = profit
        total_profit += profit
        
        # Print the detailed calculations and decisions for the current row
        print(f"Hour: {df.loc[i, 'Hour Ending']}, Real-Time Price: {real_time_price}, Day-Ahead Price: {day_ahead_price}, Spinning Reserve Price: {spinning_reserve_price}, Up Regulation Price: {up_regulation_price}")
        print(f"Charge Amount: {charge_amount}, Discharge Amount: {discharge_amount}, Spinning Reserves: {spinning_reserves}, Up Regulation: {up_regulation}")
        print(f"State of Charge (SOC): {soc}, Profit: {profit}, Total Profit: {total_profit}\n")
    
    return df, total_profit

# Apply the optimization function to the DataFrame
df, total_profit = optimize_battery(df)

# Display the total profit
print(f"Total Profit: ${total_profit:.2f}")

# Calculate and display the profits for each date
df['Date'] = pd.to_datetime(df['Date/Time']).dt.date
daily_profit = df.groupby('Date')['Profit ($)'].sum().reset_index()

print(daily_profit)


Hour: 1, Real-Time Price: 22.58, Day-Ahead Price: 21.49, Spinning Reserve Price: 3.5, Up Regulation Price: 2.28
Charge Amount: 0, Discharge Amount: 0, Spinning Reserves: 0, Up Regulation: 10
State of Charge (SOC): 8.0, Profit: 67.96, Total Profit: 67.96

Hour: 2, Real-Time Price: 21.94, Day-Ahead Price: 19.65, Spinning Reserve Price: 2.54, Up Regulation Price: 1.32
Charge Amount: 0, Discharge Amount: 0, Spinning Reserves: 0, Up Regulation: 10
State of Charge (SOC): 6.0, Profit: 57.08000000000001, Total Profit: 125.04

Hour: 3, Real-Time Price: 24.59, Day-Ahead Price: 18.71, Spinning Reserve Price: 1.5, Up Regulation Price: 1.42
Charge Amount: 0, Discharge Amount: 0, Spinning Reserves: 0, Up Regulation: 10
State of Charge (SOC): 4.0, Profit: 63.379999999999995, Total Profit: 188.42000000000002

Hour: 4, Real-Time Price: 27.43, Day-Ahead Price: 18.44, Spinning Reserve Price: 1.5, Up Regulation Price: 1.46
Charge Amount: 0, Discharge Amount: 0, Spinning Reserves: 0, Up Regulation: 10
Stat

In [19]:
df.head(50)

,Date/Time,Hour Ending,WFTANK Real-Time LMP,WFTANK Day-Ahead LMP,RRS (spinning reserve),REGUP,Charge (MWh),Discharge (MWh),Spinning Reserves (MW),SOC (MWh),Profit ($),Date,Up Regulation (MW)
0,9/10/2023,1,22.58,21.49,3.50,2.28,0,0,0,8,67.96,2023-09-10,10
1,9/10/2023,2,21.94,19.65,2.54,1.32,0,0,0,6,57.08,2023-09-10,10
2,9/10/2023,3,24.59,18.71,1.50,1.42,0,0,0,4,63.38,2023-09-10,10
3,9/10/2023,4,27.43,18.44,1.50,1.46,0,0,0,2,69.46,2023-09-10,10
4,9/10/2023,5,21.78,18.72,1.50,1.50,0,0,0,0,58.56,2023-09-10,10
5,9/10/2023,6,18.73,19.99,2.40,2.40,10,0,0,10,-187.30,2023-09-10,0
6,9/10/2023,7,18.28,18.88,1.69,1.69,0,0,0,8,53.46,2023-09-10,10
7,9/10/2023,8,16.98,18.45,1.50,1.50,0,0,0,6,48.96,2023-09-10,10
8,9/10/2023,9,15.45,17.41,1.50,1.50,0,0,0,4,45.90,2023-09-10,10
9,9/10/2023,10,14.45,16.66,3.40,3.40,0,0,0,2,62.90,2023-09-10,10
